In [1]:
# https://realpython.com/python-concurrency/

# Speed Up an I/O-Bound Program
# Synchronous Version


import requests
import time


def download_site(url, session):
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jyth

In [2]:
#Shell

$ ./io_non_concurrent.py
   [most output skipped]
Downloaded 160 in 14.289619207382202 seconds

SyntaxError: ignored

In [3]:
# threading Version


import concurrent.futures
import requests
import threading
import time


thread_local = threading.local()


def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    session = get_session()
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_site, sites)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

Read 277 from http://olympus.realpython.org/dice
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.orgRead 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org

Read 10286 from https://www.jython.org
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 10286 from https://www.jython.orgRead 277 from http://olympus.realpython.org/dice
Read 277 from http://olympus.realpython.org/dice

Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.orgRead 277 from http://olympus.realpython.org/dice

Read 10286 from https://www.jython.org
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10286 from https://www.jython.org
Read 277 from http://olympus.realpython.or

In [0]:
threadLocal = threading.local()


def get_session():
    if not hasattr(threadLocal, "session"):
        threadLocal.session = requests.Session()
    return threadLocal.session

In [7]:
#Shell

$ ./io_threading.py
   [most output skipped]
Downloaded 160 in 3.7238826751708984 seconds

SyntaxError: ignored

In [8]:
# asyncio Version

import asyncio
import time
import aiohttp


async def download_site(session, url):
    async with session.get(url) as response:
        print("Read {0} from {1}".format(response.content_length, url))


async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.ensure_future(download_site(session, url))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    asyncio.get_event_loop().run_until_complete(download_all_sites(sites))
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} sites in {duration} seconds")


ModuleNotFoundError: ignored

In [0]:
#Shell

$ ./io_asyncio.py
   [most output skipped]
Downloaded 160 in 2.5727896690368652 seconds

In [9]:
# multiprocessing Version


import requests
import multiprocessing
import time

session = None


def set_global_session():
    global session
    if not session:
        session = requests.Session()


def download_site(url):
    with session.get(url) as response:
        name = multiprocessing.current_process().name
        print(f"{name}:Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with multiprocessing.Pool(initializer=set_global_session) as pool:
        pool.map(download_site, sites)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

ForkPoolWorker-1:Read 10286 from https://www.jython.org
ForkPoolWorker-2:Read 10286 from https://www.jython.org
ForkPoolWorker-2:Read 277 from http://olympus.realpython.org/dice
ForkPoolWorker-2:Read 10286 from https://www.jython.org
ForkPoolWorker-1:Read 277 from http://olympus.realpython.org/dice
ForkPoolWorker-2:Read 277 from http://olympus.realpython.org/dice
ForkPoolWorker-1:Read 10286 from https://www.jython.org
ForkPoolWorker-2:Read 10286 from https://www.jython.org
ForkPoolWorker-1:Read 277 from http://olympus.realpython.org/dice
ForkPoolWorker-1:Read 10286 from https://www.jython.org
ForkPoolWorker-2:Read 277 from http://olympus.realpython.org/dice
ForkPoolWorker-2:Read 10286 from https://www.jython.org
ForkPoolWorker-1:Read 277 from http://olympus.realpython.org/dice
ForkPoolWorker-2:Read 277 from http://olympus.realpython.org/dice
ForkPoolWorker-1:Read 10286 from https://www.jython.org
ForkPoolWorker-2:Read 10286 from https://www.jython.org
ForkPoolWorker-1:Read 277 from htt

In [0]:
#Shell

$ ./io_mp.py
    [most output skipped]
Downloaded 160 in 5.718175172805786 seconds

In [0]:
# Speed Up a CPU-Bound Program

def cpu_bound(number):
    return sum(i * i for i in range(number))

In [12]:
# CPU-Bound Synchronous Version


import time


def cpu_bound(number):
    return sum(i * i for i in range(number))


def find_sums(numbers):
    for number in numbers:
        cpu_bound(number)


if __name__ == "__main__":
    numbers = [5_000_000 + x for x in range(20)]

    start_time = time.time()
    find_sums(numbers)
    duration = time.time() - start_time
    print(f"Duration {duration} seconds")

Duration 8.439075231552124 seconds


In [13]:
#Shell

$ ./cpu_non_concurrent.py
Duration 7.834432125091553 seconds

SyntaxError: ignored

In [0]:
# threading and asyncio Versions

#Shell

$ ./cpu_threading.py
Duration 10.407078266143799 seconds

In [14]:
# CPU-Bound multiprocessing Version

import multiprocessing
import time


def cpu_bound(number):
    return sum(i * i for i in range(number))


def find_sums(numbers):
    with multiprocessing.Pool() as pool:
        pool.map(cpu_bound, numbers)


if __name__ == "__main__":
    numbers = [5_000_000 + x for x in range(20)]

    start_time = time.time()
    find_sums(numbers)
    duration = time.time() - start_time
    print(f"Duration {duration} seconds")

Duration 10.238165616989136 seconds


In [0]:
#Shell

$ ./cpu_mp.py
Duration 2.5175397396087646 seconds